# InfCtx trainer validation

This model being trained is the small L6-D512 model with
- Layer count: 6
- Embed size: 512

The goal is to validate loss rate change, across the exact same hyper parameters with the following
- 1024 data chunk size
- same learningrate / weightdecay / seed
- "teven/enwiki_10k" dataset, chunked to 1024 token sizes

With only the change in training context size
- 1024 context vs 128 context chunks

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps
>
> And that you have completed the `baseline-setup.ipynb`

## Configure and apply your preferred settings

Adjust your desired deepspeed settings, and gpu device count.

Enable/Disable WANDB here as well ( Enabled by default, as we need the loss curve for this experiment )

( note you will need to rerun this cell, if you restart your env )

In [4]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="infctx-bptt-validation L6-D512"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto


# Baseline full context (1024) training

Perform a full 1 epoch training run of training context size = 1024. Ensuring all data samples fit within the allocated training size.

In [5]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-validation/config/L6-D512-neox-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (full, train-ctx=1024, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230730_144328-7l9nnkus
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-bptt-validation L6-D512 (full, train-ctx=1024, data-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/7l9nnkus
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_1024_bf16

# Back-Propagation Through Time (512) training

Perform a full 1 epoch training run of training context size = 512. This is a less exegerated version of the 128 training

In [6]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-validation/config/L6-D512-neox-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (bptt, train-ctx=512, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230730_150124-eu7jqsi8
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-bptt-validation L6-D512 (bptt, train-ctx=512, data-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/eu7jqsi8
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_512_bf16/b

# Back-Propagation Through Time (128) training

Perform a full 1 epoch training run of training context size = 128. Forcing all data samples to be segmented 8 times, via "Truncated Back-Propagation Through Time"

In [7]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-validation/config/L6-D512-neox-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (bptt, train-ctx=128, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.ctx_len=128

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230730_153439-ugeh2yfl
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-bptt-validation L6-D512 (bptt, train-ctx=128, data-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/ugeh2yfl
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128_bf16/b

# Last 1 segmented (128) training

Perform a full 1 epoch training run of training context size = 128. Only using the last segment. (This replicates previous known regression)

In [12]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-validation/config/L6-D512-neox-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (last-1-bptt, train-ctx=128, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.bptt_learning_range=1 \
        --model.ctx_len=128

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230731_013034-51sg7an7
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-bptt-validation L6-D512 (last-1-bptt, train-ctx=128, data-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/51sg7an7
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128

# Last 2 segmented (128) training

Last segment + 1 varient, to see the various loss learning differences

In [13]:
!cd ../../RWKV-v4neo && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python3 lightning_trainer.py fit \
        -c ../notebook/trainer-validation/config/L6-D512-neox-1024.yaml \
        --trainer.logger.init_args.name="{WANDB_PREFIX} (last-2-bptt, train-ctx=128, data-ctx=1024, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}" \
        --model.bptt_learning="true" \
        --model.bptt_learning_range=2 \
        --model.ctx_len=128

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.1.0.dev20230705'
Global seed set to 3941088705
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.7 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230731_033030-0c82dfa5
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run infctx-bptt-validation L6-D512 (last-2-bptt, train-ctx=128, data-ctx=1024, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-InfCtx-Validation
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-InfCtx-Validation/runs/0c82dfa5
Using /home/ubuntu/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ubuntu/.cache/torch_extensions/py311_cu118/wkv_128

Epoch 0: 100%|█| 5308/5308 [1:59:11<00:00,  1.35s/it, v_num=dfa5, train/loss=6.3
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:  98%|█████████████████▋| 53/54 [01:09<00:01,  1.31s/it]
Epoch 0: 100%|█| 5308/5308 [2:00:24<00:00,  1.36s/it, v_num=dfa5, train/loss=6.3
Epoch 0: 100%|█| 5308/5308 [2:00:24<00:00,  1.36s/it, v_num=dfa5, train/loss=6.3/home/ubuntu/anaconda3/envs/rwkv-exp/lib/python3.11/site-packages/torch/nn/modules/module.py:1825: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(
`Trainer.fit` stopped: `max_epochs=1` reached.
Epoch 0: 100%|█| 5308/5308 [2:00:26<00:00,  1.36s/it, v_num=dfa5, train/loss=6.3
wandb: Waiting for W&B process to finish... (success).
wandb: / 0.005 MB of 0.005 MB uploaded (0.000 MB deduped)
wandb: Run history:
wandb:              batchidx ▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███▁
wandb:          